In [1]:
import pandas as pd
import os
from mlflow.tracking import MlflowClient


from utils import get_params_and_metrics
from utils import METRICS, HYPERPARAMETERS, ARCHITECTURE, DATASETS, OTHERS

NOTEBOOK_URI="sqlite:////home/guillermo/Documents/SINC/Research/RNA/AEseq2seq/mlruns.db"
SINC_URI="sqlite:////home/gkulemeyer/Documents/Repos/seq2seq/results/mlflow/mlruns.db"

# Conectar al backend store
client = MlflowClient(tracking_uri=SINC_URI)
experiments = client.search_experiments()
for i,exp in enumerate(experiments):
    print(f"i: {i}, ID: {exp.experiment_id}, Nombre: {exp.name}, Estado: {exp.lifecycle_stage}")

i: 0, ID: 1, Nombre: pseudop_0, Estado: active
i: 1, ID: 0, Nombre: Default, Estado: active


In [2]:
pd.options.display.max_columns = 999

In [3]:
n = 0
name = experiments[n].name
id = experiments[n].experiment_id
print(f"Nombre: {name}, ID: {id}, Estado: {experiments[n].lifecycle_stage}")

df_arc = get_params_and_metrics(client, id, ARCHITECTURE  + HYPERPARAMETERS)
for col in ARCHITECTURE:
    print(f"{col}: {df_arc[col].unique()}")
for col in HYPERPARAMETERS:
    print(f"{col}: {df_arc[col].unique()}")

Nombre: pseudop_0, ID: 1, Estado: active
arc_embedding_dim: ['4']
arc_features: ['[4, 8, 16, 32, 64]' '[4, 4, 4, 4, 8]' '[4, 4, 4, 8]' '[4, 4, 8]'
 '[4, 8]' '[4, 4, 4, 4]' '[4, 4, 4]' '[4, 4]' '[4]']
arc_encoder_blocks: ['4' '3' '2' '1' '0']
arc_num_params: ['37925' '42005' '1541' '1349' '1253' '1109' '965' '869' '677' '629' '989'
 '845' '701' '605' '413' '365' '125' '653' '533' '509' '341' '293' '569'
 '425' '401' '305' '233' '185' '65']
arc_initial_volume: ['512']
arc_latent_volume: ['512.0' '64.0' '128.0' '256.0']
arc_num_conv: ['2' '1']
arc_pool_mode: ['max']
arc_up_mode: ['upsample']
arc_addition: ['cat']
arc_skip: ['0' '1']
hyp_device: ['cuda']
hyp_lr: ['0.001']
hyp_scheduler: ['none']
hyp_verbose: ['True']
max_epochs: ['50']
train_swaps: ['0']
test_swaps: ['0']


In [4]:
metr = ['test_loss', 
 'test_F1', 
 'test_Accuracy', 
 'test_Accuracy_seq']

In [5]:
n = 0
name = experiments[n].name
id = experiments[n].experiment_id
print(f"Nombre: {name}, ID: {id}, Estado: {experiments[n].lifecycle_stage}")

# METRICS, HYPERPARAMETERS, ARCHITECTURE, DATASETS, OTHERS
EXTRACT = [ 
    "arc_features",
    "arc_encoder_blocks",
    "arc_num_params",
    "arc_latent_volume",
    "arc_num_conv",
    "arc_skip", 
]

# df_metrics = get_params_and_metrics(client, id, EXTRACT + METRICS)
df_metrics = get_params_and_metrics(client, id, EXTRACT + METRICS)
print(f"df shape: {df_metrics.shape}")
df_metrics.sample(6)

Nombre: pseudop_0, ID: 1, Estado: active
df shape: (34, 19)


,run_name,arc_features,arc_encoder_blocks,arc_num_params,arc_latent_volume,arc_num_conv,arc_skip,train_loss,valid_loss,test_loss,valid_F1,train_F1,test_F1,train_Accuracy,valid_Accuracy,test_Accuracy,train_Accuracy_seq,valid_Accuracy_seq,test_Accuracy_seq
18,pseudop_nc_1_n4_3_n8_1_skip1,"[4, 4, 4, 4, 8]",4,845,64.0,1,1,0.116522,0.149622,0.134189,0.735280,0.858051,0.815254,0.830700,0.710988,0.784961,0.0,0.0,0.0
11,pseudop_nc_2_n4_3_n8_0_skip0,"[4, 4, 4, 4]",3,845,64.0,2,0,0.142128,0.171519,0.167095,0.658456,0.776799,0.717684,0.733616,0.625057,0.660349,0.0,0.0,0.0
7,pseudop_nc_2_n4_1_n8_1_skip0,"[4, 4, 8]",2,869,256.0,2,0,0.120821,0.167547,0.154563,0.698948,0.820674,0.750911,0.788516,0.675622,0.701540,0.0,0.0,0.0
3,pseudop_nc_2_n4_3_n8_1_skip0,"[4, 4, 4, 4, 8]",4,1349,64.0,2,0,0.111417,0.141640,0.127828,0.733566,0.836417,0.802643,0.807260,0.719466,0.769157,0.0,0.0,0.0
19,pseudop_nc_1_n4_3_n8_1_skip0,"[4, 4, 4, 4, 8]",4,653,64.0,1,0,0.150788,0.166054,0.146705,0.698553,0.789803,0.796662,0.733567,0.646391,0.742921,0.0,0.0,0.0
31,pseudop_nc_1_n4_1_n8_0_skip0,"[4, 4]",1,185,256.0,1,0,0.147457,0.169682,0.154445,0.703734,0.780267,0.750339,0.735585,0.665581,0.711190,0.0,0.0,0.0


In [6]:
EXPERIMENT_PATH = os.path.join("seq2pp", name)
os.makedirs(EXPERIMENT_PATH, exist_ok=True)
df_metrics.to_csv(os.path.join(EXPERIMENT_PATH, "metrics.csv"), index=False)